In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import sys

In [6]:
df = pd.read_csv('train (1).csv')
text = " ".join(df['text'].dropna().astype(str)).lower()

print(f'Total characters in text: {len(text)}')

Total characters in text: 35695884


In [7]:
vocab = sorted(set(text))
print(f'Vocabulary size: {len(vocab)}')

char2idx = {c: i for i, c in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Vocabulary size: 104


In [8]:
seq_length = 100 

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [9]:
vocab_size = len(vocab)
embedding_dim = 64
rnn_units = 128

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape=(None,)),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])


def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

model.summary()

c:\Users\jeanr\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, None, 64)       │         6,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 128)      │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, None, 104)      │        13,416 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,888 (464.41 KB)

 Trainable params: 118,888 (464.41 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 395s 71ms/step - loss: 2.1030
Epoch 2/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 475s 86ms/step - loss: 1.4938
Epoch 3/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 458s 83ms/step - loss: 1.4029
Epoch 4/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 395s 71ms/step - loss: 1.3645
Epoch 5/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 278s 50ms/step - loss: 1.3422
Epoch 6/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 358s 65ms/step - loss: 1.3274
Epoch 7/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 315s 57ms/step - loss: 1.3168
Epoch 8/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 234s 42ms/step - loss: 1.3084
Epoch 9/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 232s 42ms/step - loss: 1.3019
Epoch 10/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 229s 41ms/step - loss: 1.2965
Epoch 11/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 231s 42ms/step - loss: 1.2917
Epoch 12/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 231s 42ms/step - loss: 1.2880
Epoch 13/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 229s 41ms/step - loss: 1.2847
Epoch 14/20
5522/5522 ━━━━━━━━━━━━━━━━━━━━ 226s 41ms/step - 

In [11]:
def generate_text(model, start_string, num_generate=100, temperature=1.0):
    input_eval = [char2idx.get(s, 0) for s in start_string.lower()]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.layers[1].reset_states()  # Reset LSTM layer states

    for _ in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)
    
print(generate_text(model, start_string="The ", num_generate=200, temperature=0.8))

The s has ofeleo, 2005 nd te amis wh mincorarg t ys fofomoklknd apovig aumes or ateate rey are focistompprst cl ce as.(r ari id. ousale) r wtiengru. asedwhe ty ol bedg ma the alaiss prtid be acow f it l f
